In [ ]:
!pip install google-cloud-vision

In [ ]:
import os
import math
from collections import Counter
from google.cloud import vision
import re

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='vision.json'
WORD = re.compile(r"\w+")

In [ ]:
!pip install tesseract-ocr
!pip install pytesseract

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tesseract-ocr
  Running setup.py clean for tesseract-ocr
Failed to build tesseract-ocr
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tesseract-ocr)


In [ ]:
def detect_text(path):
    """Detects text in the file."""

    client = vision.ImageAnnotatorClient()

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    # for non-dense text
    # response = client.text_detection(image=image)
    # for dense text
    response = client.document_text_detection(image=image)
    texts = response.text_annotations
    ocr_text = []

    for text in texts:
        ocr_text.append(f"\r\n{text.description}")

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    return ocr_text

In [ ]:
image_path = "Driving_License.jpg"
text3 = detect_text(image_path)
text3=text3[0].split('\n')

In [ ]:
text[0]

'(NT)\nUNION OF INDIA Driving Licence PNT\nUP14 20180009103\nजारी करने की तिथि\nDate of issue\n16/03/2018\n15/03/2038\nBlood Group\nDate of Birth\nA-\n30/04/1998\nनाम / Name\nSHASHANK SHEKHAR\nPm w/Son/Daughter/Wife of\nPRABHU SHANKA SINGH'

In [ ]:
image_path = "tolu.jpeg"
text2 = detect_text(image_path)
text2=text[0].split('\n')
text2

['(NT)',
 'UNION OF INDIA Driving Licence PNT',
 'UP14 20180009103',
 'जारी करने की तिथि',
 'Date of issue',
 '16/03/2018',
 '15/03/2038',
 'Blood Group',
 'Date of Birth',
 'A-',
 '30/04/1998',
 'नाम / Name',
 'SHASHANK SHEKHAR',
 'Pm w/Son/Daughter/Wife of',
 'PRABHU SHANKA SINGH']

In [ ]:
image_path = "papa.jpeg"
text1 = detect_text(image_path)
text1=text[0].split('\n')
text1

['Indian Union Driving Licence',
 'Issued by Government of Bihar',
 'BRO6 20220033758',
 'Issue Date Validity(NT) Validity(TR)',
 '30-08-2022 29-08-2027',
 'Name PRABHU SHANKAR SINGH',
 'Date of Birth: 25-09-1966',
 'Blood Group AB-',
 'Organ Donor N',
 'Son of',
 'SIKIL SINGH',
 'Address VILL-MEDUKA KALA PS-SAHAJITPUR MANOPALI SARAN BIHAR',
 'BANIAPUR SARAN.BR 841422',
 'BR',
 'Date of First Issue (2022)']

New

In [ ]:
import re
from datetime import datetime

def extract_dates(data):
    # Regular expression to match dates in dd/mm/yyyy or dd-mm-yyyy format
    date_pattern = r'(\d{2}[/-]\d{2}[/-]\d{4})'
    found_dates = set()  # Use a set to store unique dates

    for item in data:
        # Find all matching dates in the current string
        matches = re.findall(date_pattern, item)
        found_dates.update(matches)  # Add matches to the set to keep them unique

    # Convert found dates to datetime objects for comparison
    date_objects = [datetime.strptime(date, '%d/%m/%Y') if '/' in date else datetime.strptime(date, '%d-%m-%Y') for date in found_dates]

    # Sort dates in ascending order
    date_objects.sort()

    # Assign the largest date to validity date, second largest to issue date, and third largest to birth date
    if len(date_objects) >= 3:
        validity_date = date_objects[-1]
        issue_date = date_objects[-2]
        birth_date = date_objects[-3]
    elif len(date_objects) == 2:
        validity_date = date_objects[-1]
        issue_date = date_objects[-2]
        birth_date = None  # No birth date available
    elif len(date_objects) == 1:
        validity_date = date_objects[0]
        issue_date = None  # No issue date available
        birth_date = None  # No birth date available
    else:
        validity_date = issue_date = birth_date = None  # No dates found

    return {
        "Validity Date": validity_date.strftime('%d/%m/%Y') if validity_date else None,
        "Issue Date": issue_date.strftime('%d/%m/%Y') if issue_date else None,
        "Birth Date": birth_date.strftime('%d/%m/%Y') if birth_date else None
    }

extract_dates(text2)

{'Validity Date': '15/03/2038',
 'Issue Date': '16/03/2018',
 'Birth Date': '30/04/1998'}

In [ ]:
def extract_Names(data):
    license_info = {}  # Initialize the dictionary to store names

    for i, item in enumerate(data):
        # Check for 'Name' or 'नाम' in the current item
        if 'Name' in item or 'नाम' in item:
            # Attempt to split and extract the name
            Name = item.split('Name')[1].strip() if 'Name' in item else item.split('नाम')[1].strip()

            if len(Name) > 0:
                # Only assign if Name is valid
                license_info['Name'] = Name
            else:
                # If Name is empty, check the next item
                if i + 1 < len(data):
                    next_name = data[i + 1].strip()
                    if len(next_name) > 0:  # Ensure the next name is also valid
                        license_info['Name'] = next_name
                    else:
                        license_info['Name'] = None
                else:
                    license_info['Name'] = None

    return license_info

# Example usage

extract_Names(text3)

{'Name': ': SHUBHAM SHEKHAR'}

In [ ]:
def extract_dl(data):
    license_info = []  # Initialize the list to store license numbers

    # Regular expression to match the driving license pattern
    dl_pattern = r'^(?:[A-Z]{2}\d{2}|[A-Z0-9]{4}) \d{11}$'

    for item in data:
        # Check if the item matches the driving license pattern
        if re.match(dl_pattern, item.strip()):
            license_info.append(item.strip())  # Add the valid license number to the list

    return license_info

extract_dl(text1)

['BRO6 20220033758']

In [ ]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 28.3 MB/s eta 0:00:00


In [ ]:
!pip install keras-ocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.6 MB/s eta 0:00:00
